#### Connect to roboflow project

In [5]:
import torch

# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# GeoTiff Images
import rasterio
from osgeo import gdal

# Visualisation
from matplotlib import pyplot as plt
import matplotlib.image as img
from matplotlib.pyplot import figure
from PIL import Image

# Model Building
import ultralytics
from ultralytics import YOLO
from IPython.display import Image

# Progress bar
from tqdm import tqdm

import numpy as np

# Others
import os
import shutil
import zipfile

%matplotlib inline

In [6]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
ultralytics.checks()

Ultralytics 8.3.225 🚀 Python-3.12.11 torch-2.7.1 CPU (Apple M1 Pro)
Setup complete ✅ (10 CPUs, 32.0 GB RAM, 714.6/1858.2 GB disk)


In [3]:
import roboflow
roboflow.login(force=True) # Login into roboflow

visit https://app.roboflow.com/auth-cli to get your authentication token.


#### Select pre-trained model

In [7]:
path_best_model = "/Users/constantin/Documents/Python_files/Machine_Learning/EY Challenge/Storm Damage - 2024/trained_model/pretrainded_pre_event_dataset_v25/content/exploratory_phase/yolo11n_training_dataset_v25/weights/best.pt"
model = YOLO(path_best_model)
model.info()

YOLO11n summary: 181 layers, 2,590,230 parameters, 0 gradients, 6.4 GFLOPs


(181, 2590230, 0, 6.4416768)

## Connect to Roboflow dataset 

In [8]:
# Downloading dataset from Roboflow platform (Notice: after execution data will be deleted from google colab)
roboflow_project_name = 'stormdamagechallenge'
dataset_name = 'storm-damage'
dataset_title = 'Storm-Damage'
dataset_id = 'goe2a'
dataset_version = 25 # Cleaned Commercial Buildings
model_name = 'yolo'
model_version = '11' # 'v8' or '11'
model_variant = 'n' # parameter required for model training phase

# Connect to roboflow dataset
rf = roboflow.Roboflow()
project = rf.workspace(roboflow_project_name).project(dataset_name + '-' + dataset_id)
dataset = project.version(dataset_version).download(model_name + model_version)

loading Roboflow workspace...
loading Roboflow project...

Version export complete for yolo11 format



Extracting Dataset Version Zip to Storm-Damage-25 in yolo11:: 100%|██████████| 1164/1164 [00:00<00:00, 7946.86it/s]


#### Read all pre-Event tile images

In [ ]:
images_path = "../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/"
pre_event_all_tiles = [images_path+filename for filename in os.listdir(images_path)]
pre_event_all_tiles

In [22]:
images_path = "Storm-Damage-25/valid/images"
# "/Users/constantin/Documents/Python_files/Machine_Learning/EY Challenge/Storm Damage - 2024/notebooks/local/Storm-Damage-25/valid/images"
validation_dataset_images = [images_path+filename for filename in os.listdir(images_path)]
validation_dataset_images

['Storm-Damage-25/valid/imagesPre_Event_tile_11_61_jpg.rf.e428c373644f8f46ab1c3e128027381a.jpg',
 'Storm-Damage-25/valid/imagesPre_Event_tile_8_20_jpg.rf.351162d10cd7599e3cd54ec04450be95.jpg',
 'Storm-Damage-25/valid/imagesPre_Event_tile_57_34_jpg.rf.9b9a69852ff8fc3ac5a37d8a7d4c1143.jpg',
 'Storm-Damage-25/valid/imagesPre_Event_tile_2_54_jpg.rf.2436538646c89b404efe75c9b0808fec.jpg',
 'Storm-Damage-25/valid/imagesPre_Event_tile_16_20_jpg.rf.ca4d8a5c93132084e1a7246fccbc5de2.jpg',
 'Storm-Damage-25/valid/imagesPre_Event_tile_15_17_jpg.rf.bba25d04f607e696abcfa24fc8b62c4d.jpg',
 'Storm-Damage-25/valid/imagesPre_Event_tile_25_71_jpg.rf.2ebd72edb8f93b2dd7e7a072c7710aa8.jpg',
 'Storm-Damage-25/valid/imagesPre_Event_tile_2_3_jpg.rf.5a2672e6fdb0909bec87a350a50e8a04.jpg',
 'Storm-Damage-25/valid/imagesPre_Event_tile_16_27_jpg.rf.2c1c02deccd15a93a62c0f91933fc356.jpg',
 'Storm-Damage-25/valid/imagesPre_Event_tile_17_15_jpg.rf.114a74a3c10d3eff30602e1a574cc137.jpg',
 'Storm-Damage-25/valid/imagesPre_

#### Define the range of tiles to annotate

In [10]:
import torch
print(torch.backends.mps.is_available())  # should be True
print(torch.backends.mps.is_built())      # should be True

True
True


In [23]:
validation_dataset_images[0]

'Storm-Damage-25/valid/imagesPre_Event_tile_11_61_jpg.rf.e428c373644f8f46ab1c3e128027381a.jpg'

In [ ]:
# list_images = ["/content/Pre_Event_tile_2_24.jpg", "/content/Pre_Event_tile_2_26.jpg"]
inference_results = model.predict(
    '/Users/constantin/Documents/Python_files/Machine_Learning/EY Challenge/Storm Damage - 2024/notebooks/local/Storm-Damage-25/valid/images', 
    save = True, # Enables saving of the annotated images or videos to file. Useful for documentation, further analysis, or sharing results. 
    save_txt = True,
    device = 'mps', # 0: GPU | 'cpu': cpu | 'mps': apple silicon mps
    show_boxes = True, # Default = True | Draws bounding boxes around detected objects. Essential for visual identification and location of objects in images or video frames.
    show_labels = False, # Default = True | Displays labels for each detection in the visual output. Provides immediate understanding of detected objects.
    show_conf = True, # Default = True | Displays the confidence score for each detection alongside the label. Gives insight into the model's certainty for each detection.
    visualize = False,
    max_det = 150,
    half = False, # Default = False |
    iou = 0.35,
    imgsz = 1024,
    conf = 0.25,
)


image 1/89 /Users/constantin/Documents/Python_files/Machine_Learning/EY Challenge/Storm Damage - 2024/notebooks/local/Storm-Damage-25/valid/images/Pre_Event_tile_0_101_jpg.rf.ec146e4529eaec213fd55c19399aa319.jpg: 1024x1024 (no detections), 76.2ms
image 2/89 /Users/constantin/Documents/Python_files/Machine_Learning/EY Challenge/Storm Damage - 2024/notebooks/local/Storm-Damage-25/valid/images/Pre_Event_tile_0_107_jpg.rf.8ffcc7c377f26366bc3b09ecef3ff9e5.jpg: 1024x1024 (no detections), 16.9ms
image 3/89 /Users/constantin/Documents/Python_files/Machine_Learning/EY Challenge/Storm Damage - 2024/notebooks/local/Storm-Damage-25/valid/images/Pre_Event_tile_0_30_jpg.rf.749560996753fc2c50c3d95fdb0dd745.jpg: 1024x1024 (no detections), 18.4ms
image 4/89 /Users/constantin/Documents/Python_files/Machine_Learning/EY Challenge/Storm Damage - 2024/notebooks/local/Storm-Damage-25/valid/images/Pre_Event_tile_0_46_jpg.rf.74787515dd653a9d0cd44e698b0d3422.jpg: 1024x1024 72 undamagedresidentialbuildings, 20.

In [20]:
inference_results[4]

ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'undamagedcommercialbuilding', 1: 'undamagedresidentialbuilding'}
obb: None
orig_img: array([[[ 97,  99, 107],
        [ 88,  90,  98],
        [ 88,  91,  96],
        ...,
        [ 76,  91, 110],
        [ 89, 105, 128],
        [110, 126, 149]],

       [[ 88,  90,  98],
        [ 82,  84,  92],
        [ 86,  89,  94],
        ...,
        [ 98, 115, 136],
        [ 99, 115, 138],
        [101, 119, 142]],

       [[ 86,  89,  97],
        [ 84,  87,  95],
        [ 91,  95, 100],
        ...,
        [114, 133, 154],
        [113, 131, 154],
        [107, 128, 150]],

       ...,

       [[240, 244, 245],
        [234, 238, 239],
        [232, 236, 237],
        ...,
        [ 56,  73,  46],
        [ 64,  81,  54],
        [ 71,  88,  61]],

       [[241, 246, 247],
        [237, 242, 243],
        [237, 242, 243],
        ...,
       

In [27]:
# Downloading dataset from Roboflow platform (Notice: after execution data will be deleted from google colab)
roboflow_project_name = 'stormdamagechallenge'
dataset_name = 'storm-damage'
dataset_id = 'goe2a'

# dataset_title = 'Storm-Damage'
# dataset_version = 20 # Cleaned Commercial Buildings
# model_name = 'yolo'
# model_version = '11' # 'v8' or '11'
# model_variant = 'n' # parameter required for model training phase

# Connect to roboflow dataset
rf = roboflow.Roboflow()
project = rf.workspace(roboflow_project_name).project(dataset_name + '-' + dataset_id)

loading Roboflow workspace...
loading Roboflow project...


In [25]:
inference_results[2].path

'../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_48_6.jpg'

In [11]:
inference_results[2].boxes

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([])
conf: tensor([])
data: tensor([], size=(0, 6))
id: None
is_track: False
orig_shape: (640, 640)
shape: torch.Size([0, 6])
xywh: tensor([], size=(0, 4))
xywhn: tensor([], size=(0, 4))
xyxy: tensor([], size=(0, 4))
xyxyn: tensor([], size=(0, 4))

In [14]:
f"batch_tiles_pre_{range_start}_{range_end}"

'batch_tiles_pre_1400_1599'

## Look for the label path

In [23]:
LABEL_PATH = f"runs/detect/predict/labels/"

In [28]:
from pathlib import Path
label_map_path = "label_mapping.txt" # The i-th line in this file contains the name of the corresponding class

for i, inference_image in enumerate(inference_results):
    image_path = inference_image.path
    image_id = image_path.split('/')[-1].split('.jpg')[0]
    label_path = LABEL_PATH + image_id + '.txt'
    label_exist = Path(label_path).exists()
    if not label_exist:
        label_path = None
    response = project.upload(
        image_path=str(image_path),
        annotation_path=label_path,
        annotation_labelmap=str(label_map_path),
        batch_name=f"batch_tiles_pre_{range_start}_{range_end}", 
        is_prediction = False, # whether the annotation data is a prediction rather than ground truth
        tag_names=['automated_labeling']
    )   

In [31]:
inference_results[12]

ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'undamagedcommercialbuilding', 1: 'undamagedresidentialbuilding'}
obb: None
orig_img: array([[[104, 103, 105],
        [122, 121, 123],
        [128, 127, 129],
        ...,
        [ 34,  26,  37],
        [ 29,  21,  31],
        [ 28,  20,  30]],

       [[116, 115, 117],
        [118, 117, 119],
        [111, 110, 112],
        ...,
        [ 32,  24,  35],
        [ 27,  19,  29],
        [ 25,  17,  27]],

       [[117, 116, 118],
        [107, 106, 108],
        [ 90,  89,  91],
        ...,
        [ 28,  20,  31],
        [ 25,  17,  27],
        [ 23,  15,  25]],

       ...,

       [[ 52,  35,  48],
        [ 51,  34,  47],
        [ 51,  34,  45],
        ...,
        [157, 163, 162],
        [158, 163, 162],
        [157, 162, 161]],

       [[ 55,  38,  51],
        [ 53,  36,  49],
        [ 53,  35,  48],
        ...,
       

In [76]:
dataset_path = "runs/detect/predict2/"


response_dataset_upload = project.upload_dataset(
    dataset_path, 
    roboflow_project_name, 
    num_workers=10, 
    batch_name=None, 
    num_retries=0, 
    is_prediction=False
)


AttributeError: 'Project' object has no attribute 'upload_dataset'

In [52]:
label_path

'runs/detect/Pre_Event_tile_32_42.txt'

In [50]:
Pre_Event_tile_32_42

NameError: name 'Pre_Event_tile_32_42' is not defined

In [42]:
from collections import defaultdict

# images_containing_commercial_buildings : dict[str] = defaultdict(str)

images_containing_commercial_buildings: dict[str, str] = {}

temp = []

for i, inference_result in enumerate(inference_results):
  classes_ids =  {class_name: class_id for class_id, class_name in inference_result.names.items()}
  image_objects_detected = inference_result.boxes.cls.cpu().numpy()
  class_id = classes_ids['undamagedcommercialbuilding']
  if class_id in image_objects_detected:
    print(f'i: {i} - {inference_result.path} - {np.count_nonzero(image_objects_detected == class_id)}')
    temp.append(inference_result.path)


i: 0 - ../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_23_15.jpg - 1
i: 2 - ../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_18_83.jpg - 1
i: 3 - ../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_37_60.jpg - 1
i: 16 - ../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_23_29.jpg - 1
i: 25 - ../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_42_8.jpg - 3
i: 27 - ../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_24_34.jpg - 1
i: 30 - ../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_39_65.jpg - 1
i: 36 - ../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_43_44.jpg - 3
i: 42 - ../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_44_59.jpg - 2
i: 46 - ../../datasets/satellite_imagery/

In [43]:
len(temp)

170

In [44]:
import os
import shutil

In [45]:
def copy_files(images_paths: list[str], destination_dir: str):

  # Create a source directory
  os.makedirs(name=destination_dir, exist_ok=True)

  for image in images_paths:
    shutil.copy2(image, destination_dir)
    print(f'Copied image \'{image}\' to directory \'{destination_dir}\'.')

In [46]:
destination_dir = '../../datasets/dataset_creation/pre_event/batch_1'
images_paths = temp

copy_files(
    images_paths = images_paths,
    destination_dir = destination_dir
)

Copied image '../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_23_15.jpg' to directory '../../datasets/dataset_creation/pre_event/batch_1'.
Copied image '../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_18_83.jpg' to directory '../../datasets/dataset_creation/pre_event/batch_1'.
Copied image '../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_37_60.jpg' to directory '../../datasets/dataset_creation/pre_event/batch_1'.
Copied image '../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_23_29.jpg' to directory '../../datasets/dataset_creation/pre_event/batch_1'.
Copied image '../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_42_8.jpg' to directory '../../datasets/dataset_creation/pre_event/batch_1'.
Copied image '../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_24_34.jpg' to director

In [33]:
path_to_copy =  '../../datasets/dataset_creation/pre_event/batch_1'

In [36]:
image_1 = '../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_23_15.jpg'
shutil.copy2(image_1, dest_path)

'../../datasets/dataset_creation/pre_event/batch_1/Pre_Event_tile_23_15.jpg'